# **Maestría en Inteligencia Artificial Aplicada**
## **Curso: Análisis de Grandes Volúmenes de Datos**
### Tecnológico de Monterrey
### Prof. Iván Olmos

## **Actividad 3**

### **Aprendizaje supervisado y no supervisado**

##### Nombre y matrícula: Mario Guillen De La Torre - A01796701


---


#### **Descripción de la Base de Datos:**

Este notebook procesa el dataset de viajes en taxi de la ciudad de Chicago, aplicando reglas de particionamiento basadas en el tipo de pago y la zona de recojo. Posteriormente se extraen submuestras representativas que serán utilizadas para analizar el comportamiento de propinas.



---

### **Marco Teórico**

#### **I. Introducción**

Los algoritmos de machine learning se dividen principalmente en dos categorías: algoritmos supervisados y no supervisados. Cada uno espera una estructura de datos distinta y resuelven problemas de diferentes naturalezas.

##### **1.1 Algoritmos Supervisados**
Estos esperan tener una variable dependiente u “objetivo” cuya relacion con las variables independientes pueda ser analizada para encontrar relaciones y generar un modelo predictivo a usar. El tipo de dato de la variable dependiente, categórica o continua, decidirá la clase de problema a abordar, clasificación o regresión. 

En los problemas de clasificación se intenta predecir la categoría discreta a la que pertenece un nuevo registro, algoritmos clásicos son:

- Clasificador de Naive Bayes
- Máquinas de Soporte Vectorial (SVMs)
- Regresión Logística
- Árboles de decisión (y sus algoritmos derivados como bosques de decisión)

La librería MLlib de PySpark tiene implementaciones de todos estos algoritmos.

En los problemas de regresión, el objetivo es predecir un valor continuo en base de las variables independientes. PySpark tiene implementaciones de los siguientes algoritmos (entre otros):

- Regresor de árbol de decisiones.
- Regresor GBT (Gradient Boosted Trees)
- Regresor FM (Factorization Machines)

##### **1.2 Algoritmos No Supervisados**
Los algoritmos no supervisados no cuentan con una variable “objetivo”, estos buscan encontrar patrones, estructuras o agrupaciones que se encuentran intrínsecamente en los datos analizados. Usos posibles de estos patrones son la toma de decisiones y reducción de dimensionalidad de los datos.

Un uso común es la búsqueda de patrones frecuentes en los datos, esto es, identificar relaciones recurrentes entre las variables. PySpark implementa dos diferentes algoritmos:

- FPGrowth
- PrefixSpan

Otro enfoque es el análisis de agrupamiento, en el cual se busca asociar cada registro a un grupo y calcular a qué grupo pertenece un registro nuevo, algoritmos comunes implementados por PySpark son:

- LDA
- GaussianMixture
- KMeans
- BisectingKMeans
- PowerIterationClustering

#### **II. Selección de los datos**
Como se mencionó en entregas anteriores, se decidieron reglas de particionamiento basadas en tres variables de caracterización, las cuales fueron seleccionadas por su relevancia en la generación de patrones comportamentales:

- payment_group: Agrupa los métodos de pago en: Credit Card, Cash, Mobile y Other. _Esta variable es fundamental, ya que existe evidencia empírica de que los pasajeros que pagan con tarjeta tienden a dejar propina con mayor frecuencia que quienes pagan en efectivo._


- pickup_zone_group: Corresponde a la zona de inicio del viaje. Se agruparon las áreas comunitarias más representativas: 76, 8, 32, 28 y Other (cualquier otra zona).
_Esta variable se usa como un proxy de contexto urbano y socioeconómico, dado que diferentes zonas pueden reflejar distintos perfiles de pasajeros._


- duration_group: Se construyó a partir de la variable duration_minutes aplicando binning basado en percentiles, con los siguientes rangos: Flash Riders  ≤10 min, Urban Cruisers entre 10 y 23.2 min y Long-Haul Nomads >23.2 min.
_Esta agrupación refleja distintos tipos de trayecto, desde viajes cortos típicos del centro urbano hasta trayectos largos, con distintas expectativas y comportamientos asociados al servicio._


Estas tres variables definen el espacio de particionamiento, generando combinaciones que capturan diferentes perfiles de pasajeros. En total, se obtienen:

- 4(payment_group) × 5(pickup_zone_group) × 3(duration_group) = **60 combinaciones de partición**

Todas las combinaciones anteriores describen a un amplio rango de viajeros, por lo que inevitablemente existen aquellas combinaciones que raramente ocurren. Para reducir la complejidad de nuestro problema se combinan las particiones que ocurren menos del 2% de las veces dentro de nuestros datos, lo que reduce el número de combinaciones a 21 (contando el nuevo grupo combinado). 

Cada una de estas particiones representa un perfil distinto de viaje (por ejemplo, trayectos largos pagados con tarjeta y partiendo de zonas turísticas). Es importante recalcar que no todos los perfiles cuentan con la misma proporción de datos, lo que podría incurrir en sesgos si la técnica de muestreo no es definida correctamente. Es por esto que se opta por un muestreo estratificado que permite extraer una proporción balanceada de los registros, tomando en cuenta todos los grupos y evitando que el modelo aprenda patrones de solo los grupos mayoritarios.

#### **REFERENCIAS**
SmartCitiesWorld. (2022). Predictive analytics key to easing traffic congestion.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://www.smartcitiesworld.net/news/news/predictive-analytics-key-to-easing-traffic-congestion-7502

Guo, Y., Liu, Y., Wang, J., & Chen, H. (2023). Urban mobility hotspots and their implications for resilient city planning.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Journal of Transport Geography, 108, 103567. https://www.sciencedirect.com/science/article/pii/S096669232300039X?via%3Dihub

Le, James. (2019, Julio 23). Using Ant Colony and Genetic Evolution to Optimize Ride-Sharing Trip Duration.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Medium. https://medium.com/data-science/using-ant-colony-and-genetic-evolution-to-optimize-ride-sharing-trip-duration-56194215923f

City of Chicago. (2024). Taxi Trips (2024-) [Conjunto de datos].  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://data.cityofchicago.org/Transportation/Taxi-Trips-2024-/ajtu-isnz/about_data

Ahmed, S. K. (2024). Research methodology simplified: how to choose the right sampling technique and determine the appropriate sample size for research.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Oral Oncology Reports, 12, 100662. https://doi.org/10.1016/j.oor.2024.100662

Polak, A. (2023). Scaling Machine Learning with Spark: Distributed ML with MLlib, TensorFlow, and Pytorch. O’Reilly Media.

---

### **Implementación**

#### **Importación de Librerías**

Como primer paso importamos las librerías que serán necesarias para la ejecución de nuestro código.

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Imputer,StringIndexer,OneHotEncoder,StandardScaler,VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col, isnan, when, count, percentile_approx, min, max, mean, stddev, approx_count_distinct, expr,  concat_ws, lit
from pyspark.sql.functions import hour, dayofweek, unix_timestamp, when, month,to_timestamp,  dayofweek
from pyspark.sql import functions as F

from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator, ClusteringEvaluator
from pyspark.ml.clustering import KMeans

import scipy.stats as stats
import pandas as pd
import os
from IPython.display import display, HTML

#### **Creación de la Sesión Spark**

Posteriormente, generamos nuestra sesión de Spark y definimos una función que me permitirá el imprimir los DataFrames de Spark en una vista mucho más amigable, para esto el DataFrame se convierte a un pandas DataFrame y se imprime usando HTML. 

In [2]:
spark = SparkSession.builder \
    .appName("ChicagoTaxyTripsAnalysis") \
     .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.python.worker.timeout", "600") \
    .config("spark.python.worker.retries", "3") \
    .getOrCreate()

In [3]:
def pretty_display(df, limit=100):
    pdf = df.limit(limit).toPandas()
    display(HTML(pdf.to_html(notebook=True)))

#### **Carga del Dataset**

Ahora cargamos nuestro dataset y observamos el número de registros y columnas, dando una idea de la dimensión de los datos.

In [4]:
filename = "Taxi_Trips__2024-__20250426.csv"
local_path = f"C:/Users/mario/Maestria/Grandes Cantidades de Datos/{filename}"
dftaxytrips = spark.read.csv(local_path, header=True, inferSchema=True)

In [5]:
print("Número de registros:", dftaxytrips.count())
print("Número de columnas:", len(dftaxytrips.columns))

Número de registros: 7917844
Número de columnas: 23


#### **Exploración de los Datos**

A continuación podemos verificar el Schema de nuestro dataset, haciendo nota de las columnas que son de tipo textual, ya que estas recibirán un trato distinto en las siguientes secciones.

In [6]:
dftaxytrips.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

#### **Variables de Caracterización**

Como se mencionó en entregas anteriores, para nuestro análisis se decidió realizar transformaciones a nuestros datos para generar nuevas columnas que puedan presentar con información significativa para nuestro análisis. Un ejemplo claro de esto es la creación de las columnas "trip_hour","trip_day_of_week"y"trip_month" las cuales desglosan la fecha y hora del inicio de un viaje y podrían ser de gran importancia para encontrar relaciones en nuestros datos.

Además, se agrupan valores en ciertas columnas y se realiza binning en otras para reducir la complejidad de nuestros datos.  

In [7]:
# Trip Start Timestamp es tipo timestamp
dftaxytrips = dftaxytrips.withColumn(
    "trip_start_ts",
    to_timestamp(col("Trip Start Timestamp"), "MM/dd/yyyy hh:mm:ss a")
)

# Hora del día
dftaxytrips = dftaxytrips.withColumn("trip_hour", hour(col("trip_start_ts")))

# Día de la semana (1 = domingo, 7 = sábado)
dftaxytrips = dftaxytrips.withColumn("trip_day_of_week", dayofweek(col("trip_start_ts")))

# Mes del año (1 = enero, 12 = diciembre)
dftaxytrips = dftaxytrips.withColumn("trip_month", month(col("trip_start_ts")))

# Duración del viaje en minutos
dftaxytrips = dftaxytrips.withColumn("duration_minutes", col("Trip Seconds") / 60)

# Tip/Fare ratio
dftaxytrips = dftaxytrips.withColumn("tip_ratio",
    when(col("Fare") > 0, col("Tips") / col("Fare")).otherwise(0))

# Tip/Trip Miles ratio
dftaxytrips = dftaxytrips.withColumn("tip_per_mile",
    when(col("Trip Miles") > 0, col("Tips") / col("Trip Miles")).otherwise(0))

# Agrupación método de pago
dftaxytrips = dftaxytrips.withColumn("payment_group",
    when(col("Payment Type") == "Credit Card", "Credit Card")
    .when(col("Payment Type") == "Cash", "Cash")
    .when(col("Payment Type") == "Mobile", "Mobile")
    .otherwise("Other"))

# Agrupación de Compañia
dftaxytrips = dftaxytrips.withColumn("company_group",
    when(col("Company") == "Flash Cab", "Flash Cab")
    .when(col("Company") == "Taxi Affiliation Services", "Taxi Affiliation")
    .when(col("Company") == "Taxicab Insurance Agency Llc", "Insurance Agency")
    .when(col("Company") == "Sun Taxi", "Sun Taxi")
    .when(col("Company") == "City Service", "City Service")
    .when(col("Company") == "Chicago Independents", "Chicago Independents")
    .otherwise("Other"))

# Agrupación Zona origen
dftaxytrips = dftaxytrips.withColumn("pickup_zone_group",
    when(col("Pickup Community Area") == 76, 76)
    .when(col("Pickup Community Area") == 8, 8)
    .when(col("Pickup Community Area") == 32, 32)
    .when(col("Pickup Community Area") == 28, 28)
    .otherwise("Other"))

# Agrupación Zona destino
dftaxytrips = dftaxytrips.withColumn("dropoff_zone_group",
    when(col("Dropoff Community Area") == 8, 8)
    .when(col("Dropoff Community Area") == 32, 32)
    .when(col("Dropoff Community Area") == 28, 28)
    .when(col("Dropoff Community Area") == 76, 76)
    .otherwise("Other"))

# Renombrar ciertas columnas
dftaxytrips = dftaxytrips.withColumnRenamed("Trip ID", "trip_id")
dftaxytrips = dftaxytrips.withColumnRenamed("Trip Miles", "trip_miles")

In [8]:
# Duración del viaje (en minutos)
dftaxytrips = dftaxytrips.withColumn(
    "duration_group",
    (
        when(col("duration_minutes") <= 10.0, "Flash Riders")           # viajes muy cortos, de alta rotación
        .when(col("duration_minutes") <= 23.2, "Urban Cruisers")        # trayectos típicos dentro de la ciudad
        .otherwise("Long-Haul Nomads")                                  # trayectos largos, posiblemente entre distritos lejanos
    )
)

#### **Regla de Particionamiento**

Debido a que el objetivo principal del proyecto está enfocado en analizar las propinas (tips) en los viajes de taxi de Chicago y predecir patrones relevantes, hemos considerado estas tres variables para realizar nuestro particionamiento:


*   **`payment_group`:** Agrupación del método de pago Credit Card, Cash, Mobile, Other. Se considera clave debido a la fuerte relación entre pagos con tarjeta y la propina otorgada.
*   **`pickup_zone_group`:** 	Agrupación de zonas de recojo, áreas específicas (76, 8, 32, 28) y un grupo "Other" que incluye las demás zonas. Representa un proxy de ubicación socioeconómica o comercial.
*   **`duration_group`:** Clasificación de duración del viaje Flash Riders (≤10 min), Urban Cruisers (10–23.2 min), Long-Haul Nomads (>23.2 min). Captura la intensidad y contexto del trayecto.

Asimismo, consideramos que estas variables permiten capturar factores clave de comportamiento relacionados con la decisión del pasajero de dejar una propina.

A continuación realizamos el particionamiento de nuestros datos utilizando las variables mencionadas anteriormente, esto nos indicará la proporción de cada segmento ayudándonos a tomar decisiones en nuestro muestreo. 

In [9]:
partition_counts = dftaxytrips.groupBy(
    "payment_group", "pickup_zone_group", "duration_group"
).agg(count("*").alias("count"))

# Calcular total general
total_count = dftaxytrips.count()

# Agregar proporción por combinación
partition_counts = partition_counts.withColumn(
    "proportion", col("count") / total_count
)

# Ordenar por las más representativas
partition_counts.orderBy(col("proportion").desc()).show(60)


+-------------+-----------------+----------------+------+--------------------+
|payment_group|pickup_zone_group|  duration_group| count|          proportion|
+-------------+-----------------+----------------+------+--------------------+
|  Credit Card|               76|Long-Haul Nomads|932993| 0.11783422355883748|
|        Other|            Other|  Urban Cruisers|447407| 0.05650616506210529|
|        Other|            Other|Long-Haul Nomads|430338| 0.05435040144766681|
|         Cash|            Other|    Flash Riders|310727| 0.03924389012968682|
|         Cash|                8|    Flash Riders|306484| 0.03870801192849973|
|  Credit Card|               32|    Flash Riders|291907|0.036866980455790746|
|  Credit Card|                8|    Flash Riders|276498| 0.03492086987316244|
|         Cash|               32|    Flash Riders|255828| 0.03231031073610442|
|       Mobile|                8|    Flash Riders|225234|0.028446380100441485|
|  Credit Card|            Other|Long-Haul Nomads|21

Como podemos ver, tenemos un total de 60 segmentos en nuestra población, muchos de estos cuentan con una cantidad muy reducida de datos, por lo que incluirlos en nuestras técnicas de muestreo puede propiciar errores y sobre complicar el proceso para segmentos muy poco representativos de la población en general. Por estas razones se opta por conglomerar aquellos que cuentan con menos de un 2% de la población en un nuevo segmento considerado "Other".

Para esto se realizan los siguientes pasos:
- Se crea un dataset con solo los segmentos que tienen más del 2% de la población utilizando un filtro en la columna "proportion"
- A dicho dataset se le agrega una columna "StrataGrouping" que funcionará como identificador concatenando los valores de "payment_group", "pickup_zone_group", "duration_group"
- Se hace una unión al dataset original usando las tres columnas antes mencionadas, esto hace que la columna "StrataGrouping" tenga solo valores en aquellos segmentos con más del 2% de la población.
- Para el resto de los segmentos se imputa el valor "Other" 

In [10]:
significant_combinations = partition_counts.filter(col("proportion") > 0.02)

significant_combinations = significant_combinations.withColumn(
    "StrataGrouping", concat_ws("_", "payment_group", "pickup_zone_group", "duration_group")
)

dftaxytrips_with_strata = dftaxytrips.join(
    significant_combinations.select(
        "payment_group", "pickup_zone_group", "duration_group", "StrataGrouping"
    ),
    on=["payment_group", "pickup_zone_group", "duration_group"],
    how="left"
)

dftaxytrips_with_strata = dftaxytrips_with_strata.withColumn(
    "StrataGrouping",
    when(col("StrataGrouping").isNull(), lit("Other")).otherwise(col("StrataGrouping"))
)

In [11]:
grouped_partition_counts = dftaxytrips_with_strata.groupBy(
    "StrataGrouping"
).agg(count("*").alias("count"))

# Agregar proporción por combinación
grouped_partition_counts = grouped_partition_counts.withColumn(
    "proportion", col("count") / total_count
)

# Ordenar por las más representativas
grouped_partition_counts.orderBy(col("proportion").desc()).show(60)


+--------------------+-------+--------------------+
|      StrataGrouping|  count|          proportion|
+--------------------+-------+--------------------+
|               Other|2377353|  0.3002525687548277|
|Credit Card_76_Lo...| 932993| 0.11783422355883748|
|Other_Other_Urban...| 447407| 0.05650616506210529|
|Other_Other_Long-...| 430338| 0.05435040144766681|
|Cash_Other_Flash ...| 310727| 0.03924389012968682|
| Cash_8_Flash Riders| 306484| 0.03870801192849973|
|Credit Card_32_Fl...| 291907|0.036866980455790746|
|Credit Card_8_Fla...| 276498| 0.03492086987316244|
|Cash_32_Flash Riders| 255828| 0.03231031073610442|
|Mobile_8_Flash Ri...| 225234|0.028446380100441485|
|Credit Card_Other...| 211508|0.026712827380786994|
|Cash_76_Long-Haul...| 206361|0.026062776685168335|
|Cash_Other_Long-H...| 205220| 0.02591867180005062|
|Mobile_Other_Urba...| 203128|0.025654458461166953|
|Credit Card_76_Ur...| 199533|  0.0252004207205901|
|Cash_Other_Urban ...| 194488| 0.02456325231969713|
|Credit Card

#### **Técnica de Muestreo**

Ahora que se tiene la columna "StrataGrouping" podemos usar muestreo estratificado, para esto primero creamos un diccionario que tome los valores únicos de "StrataGrouping" y les asigne el porcentaje de valores que se tomaría de la población de dicho segmento, después se utiliza la función "SampleBy" para realizar la extracción de los datos.

En nuestro caso se optó por utilizar el 15% de los datos de cada estrato, esto nos permite ahorrar una cantidad considerable de recursos de procesamiento y almacenamiento, mientras que mantiene la distribución de los datos sin afectar a ningún estrato, manteniendo la integridad de nuestra población en la muestra.

In [12]:
fractions_df = dftaxytrips_with_strata.select("StrataGrouping").distinct().withColumn("fraction",lit(0.15))
fractions_dict = fractions_df.rdd.collectAsMap()

In [13]:
sampled_df = dftaxytrips_with_strata.stat.sampleBy("StrataGrouping", fractions_dict, seed=42)

In [14]:
sampled_total_count = sampled_df.count()

In [15]:
sampled_partition_counts = sampled_df.groupBy(
    "StrataGrouping"
).agg(count("*").alias("count"))

# Agregar proporción por combinación
sampled_partition_counts = sampled_partition_counts.withColumn(
    "proportion", col("count") / sampled_total_count 
)

# Ordenar por las más representativas
sampled_partition_counts.orderBy(col("proportion").desc()).show(60)

+--------------------+------+--------------------+
|      StrataGrouping| count|          proportion|
+--------------------+------+--------------------+
|               Other|356854| 0.30022976553146846|
|Credit Card_76_Lo...|140422| 0.11814037151176633|
|Other_Other_Urban...| 67046| 0.05640739590931539|
|Other_Other_Long-...| 64443| 0.05421743004182221|
|Cash_Other_Flash ...| 46794| 0.03936890618650635|
| Cash_8_Flash Riders| 46027| 0.03872361082716433|
|Credit Card_32_Fl...| 44029|0.037042645862411586|
|Credit Card_8_Fla...| 41543| 0.03495111488024176|
|Cash_32_Flash Riders| 38282| 0.03220755794828046|
|Mobile_8_Flash Ri...| 33494|  0.0281792995642784|
|Credit Card_Other...| 31932|0.026865151779021254|
|Cash_Other_Long-H...| 30787| 0.02590183602094223|
|Mobile_Other_Urba...| 30682|0.025813497021293066|
|Cash_76_Long-Haul...| 30650|0.025786574659495222|
|Credit Card_76_Ur...| 29751|0.025030224557737107|
|Cash_Other_Urban ...| 29284|0.024637326340249857|
|Credit Card_8_Urb...| 27575|0.

In [16]:
print(sampled_total_count)

1188603


In [17]:
print(total_count)

7917844


#### **Preprocesamiento**

##### **Eliminación de columnas no importantes**

In [18]:
# Estructura del dataset
sampled_df.printSchema()

root
 |-- payment_group: string (nullable = false)
 |-- pickup_zone_group: string (nullable = false)
 |-- duration_group: string (nullable = false)
 |-- trip_id: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable 

En nuestro dataset podemos observar que existen múltiples columnas dedicadas a representar datos de localización, esto complica nuestro modelo sin aportar información significativa, por lo que eliminaremos todas menos 'pickup_zone_group' y 'dropoff_zone_group', las cuales construimos con anterioridad por esta misma razón.

Además, la columna 'Trip Minutes' se calculó directamente de la columna 'Trip Seconds', por lo que eliminamos esta última para evitar problemas de colinealidad. Las columnas 'trip_id' y 'taxi_id' no presentan valores importantes para nuestro análisis, por lo que se pueden eliminar. Y las columnas 'Trip Start Timestamp', 'Trip End Timestamp' y 'trip_start_ts' presentan un problema similar a las anteriores, donde sus valores están representados en otras columnas o no contribuyen con información importante para nuestro análisis, por lo que las podemos eliminar. 

In [19]:
sampled_df = sampled_df.drop('Pickup Census Tract','Dropoff Census Tract','Pickup Community Area','Dropoff Community Area','Pickup Centroid Longitude','Pickup Centroid Latitude','Dropoff Centroid Latitude','Pickup Centroid Location','Dropoff Centroid Longitude','Dropoff Centroid  Location','Taxi ID','trip_id','Trip Seconds','Trip Start Timestamp', 'Trip End Timestamp','trip_start_ts','Payment Type','Company','StrataGrouping')

##### **Manejo de datos faltantes**

In [20]:
missing_taxytrips = sampled_df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in sampled_df.columns
])

In [21]:
print("Valores faltantes en nuestra muestra:")
pretty_display(missing_taxytrips)

Valores faltantes en nuestra muestra:


,payment_group,pickup_zone_group,duration_group,trip_miles,Fare,Tips,Tolls,Extras,Trip Total,trip_hour,trip_day_of_week,trip_month,duration_minutes,tip_ratio,tip_per_mile,company_group,dropoff_zone_group
0,0,0,0,8,3049,3049,3049,3049,3049,0,0,0,244,0,2790,0,0


Debido a que nuestro objetivo es predecir los valores de Tip, es importante eliminar aquellos casos en los cuales esta columna tenga valores nulos.

In [22]:
sampled_df = sampled_df.where(sampled_df.Tips != 0)

Para las columnas numéricas restantes podemos definir un imputador simple que use el promedio de cada columna.

In [23]:
# Lista de variables numéricas a imputar
vars_a_imputar = ["duration_minutes", "trip_miles", "Fare","Tolls","Extras","Trip Total","tip_ratio", "tip_per_mile"]

# Aplicamos imputación con mediana
for var in vars_a_imputar:
    mediana = sampled_df.approxQuantile(var, [0.5], 0.01)[0]
    sampled_df = sampled_df.withColumn(
        var, when(col(var).isNull(), mediana).otherwise(col(var))
    )

En el caso de las variables categóricas definimos un imputador que use un valor comodín para asegurarnos que no existan valores nulos en el futuro.

Mientras que las variables "trip_hour","trip_day_of_week" y "trip_month" son de tipo numéricas, estas representan valores categóricos (hora, día de la semana y mes) por lo que las consideraré como variables categóricas y tendrán un imputador separado.

In [24]:
# Lista de variables numéricas a imputar
vars_a_imputar = ["payment_group", "pickup_zone_group", "duration_group","company_group","dropoff_zone_group"]

# Aplicamos imputación con mediana
for var in vars_a_imputar:
    sampled_df = sampled_df.withColumn(
        var, when(col(var).isNull(), "NA").otherwise(col(var))
    )
vars_a_imputar = ["trip_hour","trip_day_of_week","trip_month"]

# Aplicamos imputación con mediana
for var in vars_a_imputar:
    sampled_df = sampled_df.withColumn(
        var, when(col(var).isNull(), 0).otherwise(col(var))
    )

In [25]:
# Análisis de valores faltantes en 'dftaxytrips_selected'
missing_taxytrips = sampled_df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in sampled_df.columns
])

In [26]:
print("Valores faltantes en Chicago Taxi Trips Dataset (csv):")
pretty_display(missing_taxytrips)

Valores faltantes en Chicago Taxi Trips Dataset (csv):


,payment_group,pickup_zone_group,duration_group,trip_miles,Fare,Tips,Tolls,Extras,Trip Total,trip_hour,trip_day_of_week,trip_month,duration_minutes,tip_ratio,tip_per_mile,company_group,dropoff_zone_group
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
sampled_df.printSchema()

root
 |-- payment_group: string (nullable = false)
 |-- pickup_zone_group: string (nullable = false)
 |-- duration_group: string (nullable = false)
 |-- trip_miles: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- trip_hour: integer (nullable = true)
 |-- trip_day_of_week: integer (nullable = true)
 |-- trip_month: integer (nullable = true)
 |-- duration_minutes: double (nullable = true)
 |-- tip_ratio: double (nullable = true)
 |-- tip_per_mile: double (nullable = true)
 |-- company_group: string (nullable = false)
 |-- dropoff_zone_group: string (nullable = false)



##### **Manejo de valores atípicos**

Primero definimos funciones auxiliares que nos ayudaran a detectar y eliminar valores atípicos. Es importante considerar que aunque no se detecten valores atípicos en la muestra actual, se incluyo una sección donde estos se eliminan para prevenirnos a su ocurrencia en otras muestras. 

In [28]:
def count_outliers(df, column):
    percentiles = df.approxQuantile(column, [0.25, 0.75], 0.05) 
    Q1 = percentiles[0]
    Q3 = percentiles[1]

    IQR = Q3 - Q1

    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

    return df.filter((col(column) < lower_limit) & (col(column) > upper_limit)).count()

def remove_outliers_inplace(df, column):
    
    percentiles = df.approxQuantile(column, [0.25, 0.75], 0.05) 
    Q1 = percentiles[0]
    Q3 = percentiles[1]

    IQR = Q3 - Q1

    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

    df = df.filter((col(column) >= lower_limit) & (col(column) <= upper_limit))

    return df

In [29]:
NumVar = ["duration_minutes", "trip_miles", "Fare","Tips","Tolls","Extras","Trip Total","tip_ratio", "tip_per_mile"]
for i in NumVar:
    outlier_count = count_outliers(sampled_df, i)
    print(f"Valores atípicos para {i}: {outlier_count}")

Valores atípicos para duration_minutes: 0
Valores atípicos para trip_miles: 0
Valores atípicos para Fare: 0
Valores atípicos para Tips: 0
Valores atípicos para Tolls: 0
Valores atípicos para Extras: 0
Valores atípicos para Trip Total: 0
Valores atípicos para tip_ratio: 0
Valores atípicos para tip_per_mile: 0


Agregamos código que maneje los valores atípicos en caso de que ocurran  

In [30]:
sampled_no_outlier_df = sampled_df
for i in NumVar:
    sampled_no_outlier_df = remove_outliers_inplace(sampled_df, i)

#### **Preparación del conjunto de entrenamiento y prueba**

Para evitar problemas de inyección de sesgos se hace la separación del conjunto de entrenamiento y prueba.

In [31]:
train_df, test_df = sampled_no_outlier_df.randomSplit([0.7, 0.3], seed=42)

##### **Transformación de datos categóricos**

Los modelos de PySpark no permiten el uso de columnas que no sean de tipo numérico, por lo que las siguientes columnas tendrán que ser transformadas utilizando un "StringIndexer", además, se utilizara la técnica de one hot encoding para evitar que se infieran relaciones numéricas donde no las hay. 

In [32]:
indexer = StringIndexer(
            inputCols=["payment_group","company_group","pickup_zone_group","dropoff_zone_group","duration_group"], 
            outputCols= ["payment_group_cat","company_group_cat","pickup_zone_group_cat","dropoff_zone_group_cat","duration_group_cat"])
indexerFit = indexer.fit(train_df)
train_indexed_df = indexerFit.transform(train_df)

In [33]:
train_indexed_df = train_indexed_df.select(
 'trip_miles',
 'Fare',
 'Tips',
 'Tolls',
 'Extras',
 'Trip Total',
 'trip_hour',
 'trip_day_of_week',
 'trip_month',
 'duration_minutes',
 'tip_ratio',
 'tip_per_mile',
 'payment_group_cat',
 'company_group_cat',
 'pickup_zone_group_cat',
 'dropoff_zone_group_cat',
 'duration_group_cat')

In [34]:
encoder = OneHotEncoder(
            inputCols=["payment_group_cat","company_group_cat","pickup_zone_group_cat","dropoff_zone_group_cat","duration_group_cat","trip_hour", "trip_day_of_week", "trip_month"], 
            outputCols= ["payment_group_ohe","company_group_ohe","pickup_zone_group_ohe","dropoff_zone_group_ohe","duration_group_ohe","trip_hour_ohe", "trip_day_of_week_ohe", "trip_month_ohe"])
encoderFit = encoder.fit(train_indexed_df)
train_encoded_df = encoderFit.transform(train_indexed_df)

In [35]:
train_encoded_df = train_encoded_df.select(
 'trip_miles',
 'Fare',
 'Tips',
 'Tolls',
 'Extras',
 'Trip Total',
 'trip_hour_ohe',
 'trip_day_of_week_ohe',
 'trip_month_ohe',
 'duration_minutes',
 'tip_ratio',
 'tip_per_mile',
 'payment_group_ohe',
 'company_group_ohe',
 'pickup_zone_group_ohe',
 'dropoff_zone_group_ohe',
 'duration_group_ohe')

##### **Transformación de datos numéricos para modelo de aprendizaje supervisado**

Debido a que los datos numéricos se presentan en distintas escalas y a que no existen valores atípicos (nos encargamos de estos en pasos anteriores), se optó por utilizar un standard scaler para transformar estas columnas. 

Es importante señalar que no se modifica la columna de Tip, ya que es nuestra columna objetivo para el algoritmo de aprendizaje supervisado. 

In [36]:
numeric_cols = ["duration_minutes", "trip_miles", "Fare", "Tolls", "Extras", "Trip Total", "tip_ratio", "tip_per_mile"]
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="numeric_features_vec")
train_encoded_num_vectorized_df = assembler.transform(train_encoded_df)

In [37]:
scaler = StandardScaler(
    inputCol="numeric_features_vec",
    outputCol="numeric_features_scaled",
    withMean=True, 
    withStd=True
)

scalerFit = scaler.fit(train_encoded_num_vectorized_df)  
train_encoded_num_scaled_df = scalerFit.transform(train_encoded_num_vectorized_df)

##### **DataFrame final para modelo de aprendizaje supervisado**

Finalmente, se genera el DataFrame final para el modelo de aprendizaje supervisado. Para esto se utiliza un "VectorAssembler" que genere la columna "features" la cual contendrá vectores con todas las columnas anteriormente transformadas.  

In [38]:
final_features = ["numeric_features_scaled", 'payment_group_ohe',
 'company_group_ohe',
 'pickup_zone_group_ohe',
 'dropoff_zone_group_ohe',
 'duration_group_ohe','trip_hour_ohe',
 'trip_day_of_week_ohe']  
assembler_final = VectorAssembler(inputCols=final_features, outputCol="features")
train_final_df = assembler_final.transform(train_encoded_num_scaled_df )

In [39]:
train_final_df = train_final_df.select('features','tips')
train_final_df = train_final_df.withColumnRenamed("tips", "label")

##### **Transformando el set de testing para modelo de aprendizaje supervisado**

Además, transformamos nuestro dataset de prueba, cuidando de no utilizar la función fit en ninguno de nuestros transformadores. 

In [40]:
test_indexed_df = indexerFit.transform(test_df)
test_encoded_df = encoderFit.transform(test_indexed_df)
test_vectorized_df = assembler.transform(test_encoded_df)
test_scaled_df = scalerFit.transform(test_vectorized_df)
test_final_df = assembler_final.transform(test_scaled_df)
test_final_df = test_final_df.select('features','tips')

In [41]:
test_final_df  = test_final_df .withColumnRenamed("tips", "label")

##### **Transformación de datos numéricos para modelo de aprendizaje no supervisado**

A continuación se repiten los pasos anteriores para el dataset del modelo de aprendizaje no supervisado, la diferencia principal siendo que este incluye la columna "tips".

In [42]:
numeric_cols_nonSup = ["duration_minutes", "trip_miles", "Fare", "Tolls", "Extras", "Trip Total", "tip_ratio", "tip_per_mile","tips"]
assembler_nonSup = VectorAssembler(inputCols=numeric_cols, outputCol="numeric_features_vec")
train_vectorized_df_nonSup = assembler.transform(train_encoded_df)

In [43]:
scaler_nonSup = StandardScaler(
    inputCol="numeric_features_vec",
    outputCol="numeric_features_scaled",
    withMean=True, 
    withStd=True
)

scalerFit_nonSup = scaler.fit(train_vectorized_df_nonSup)  
train_scaled_df_nonSup = scalerFit.transform(train_vectorized_df_nonSup)

##### **DataFrame final para modelo de aprendizaje no supervisado**

In [44]:
train_final_df_nonSup = assembler_final.transform(train_scaled_df_nonSup )

In [45]:
train_final_df_nonSup = train_final_df.select('features')

##### **Transformando el set de testing para modelo de aprendizaje no supervisado**

In [46]:
test_vectorized_nonSup_df = assembler_nonSup.transform(test_encoded_df)
test_scaled_nonSup_df = scalerFit_nonSup.transform(test_vectorized_df)
test_final_nonSup_df = assembler_final.transform(test_scaled_df)
test_final_nonSup_df = test_final_nonSup_df .select('features')

#### **Creación de modelo de aprendizaje supervisado**

Para nuestro problema que utilice aprendizaje supervisado, utilizaremos regresión lineal para predecir los valores de la columna 'tips'.

Definimos nuestro modelo y los parámetros a usar en nuestro grid search

In [47]:
lr = LinearRegression()
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.0, 0.01, 0.1]).addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]).build()
evaluator = RegressionEvaluator(metricName="rmse") 
cv = CrossValidator(
    estimator=lr,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3, 
    parallelism=4
)

Entrenamos nuestro grid search y obtenemos el mejor modelo

In [48]:
cv_model = cv.fit(train_final_df)
best_model = cv_model.bestModel

Imprimimos los parametros de nuestro modelo y verificamos el rendimiento en nuestro set de datos de prueba

In [49]:
predictions = best_model.transform(test_final_df)
rmse = evaluator.evaluate(predictions)

print("Parametros de mejor modelo:")
print(f"  regParam: {best_model._java_obj.getRegParam()}")
print(f"  elasticNetParam: {best_model._java_obj.getElasticNetParam()}")
print(f"RMSE en set de prueba: {rmse}")

Parametros de mejor modelo:
  regParam: 0.0
  elasticNetParam: 0.0
RMSE en set de prueba: 0.3389000723663502


In [50]:
predictions.select("label", "prediction").show(5)

+-----+------------------+
|label|        prediction|
+-----+------------------+
|  2.0|2.1460191983007038|
|  3.0| 3.113884371945795|
|  1.0|1.1283108188735556|
|  2.0| 2.128203154836537|
|  9.5| 9.448396205437547|
+-----+------------------+
only showing top 5 rows



Mostramos métricas de nuestra columna objetivo para contextualizar nuestros resultados

In [51]:
train_final_df.selectExpr("mean(label)", "stddev(label)", "min(label)", "max(label)").show()

+------------------+-----------------+----------+----------+
|       mean(label)|    stddev(label)|min(label)|max(label)|
+------------------+-----------------+----------+----------+
|6.0354008949752345|4.440883954488067|      0.01|     150.0|
+------------------+-----------------+----------+----------+



#### **Creación de modelo de aprendizaje no supervisado**

Para el caso del modelo de aprendizaje no supervisado, se optó por utilizar KMeans para realizar agrupamientos en nuestra población. Esto podría de ser de gran utilidad comercial para comprender mejor los patrones de comportamiento de los consumidores.

Definimos los valores a utilizar nuestro GridSearch y el evaluador

In [52]:
k_values = [2, 4, 6, 8]
max_iter_values = [10, 20]
evaluator = ClusteringEvaluator()

Realizamos el GridSearch, como PySpark no cuenta con una implementación nativa, utilizaremos dos fors anidados. 

In [53]:
best_model = None
best_score = float('-inf')
best_params = {}

for k in k_values:
    for max_iter in max_iter_values:
        kmeans = KMeans(k=k, maxIter=max_iter, seed=1)
        model = kmeans.fit(train_final_df_nonSup)
        predictions = model.transform(train_final_df_nonSup)
        
        score = evaluator.evaluate(predictions) 
        
        if score > best_score:
            best_score = score
            best_model = model
            best_params = {"k": k, "maxIter": max_iter}

In [54]:
print(f"Mejores Parametros={best_params}, Mejor Resultado={best_score}")

Mejores Parametros={'k': 2, 'maxIter': 10}, Mejor Resultado=0.4181930478893609


Tomamos el mejor modelo y lo probamos con nuestros datos de prueba

In [55]:
testResults = evaluator.evaluate(best_model.transform(test_final_nonSup_df))
print(f"Resultado del modelo con datos de prueba = {testResults:.4f}")

Resultado del modelo con datos de prueba = 0.4313
